In [1]:
import os
import random
import skimage.data
from skimage import io
import skimage.transform as skt
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

In [2]:
# submap_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhuan/submap"
# pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhuan/picture"

# # for i in range(4):
# i = 1
# submap1 = cv2.imread(os.path.join(submap_dir,str(0) + '.pgm'))
# submap1 = cv2.resize(submap1,(448,448))
# submap1 = cv2.transpose(submap1)
# submap1 = cv2.flip(submap1, 1)
# # submap1 = np.rot90(submap1)
# cv2.imshow("submap",submap1)


# pic1 = cv2.imread(os.path.join(pic_dir, '00000.jpg'))
# cv2.imshow("pic",pic1)

# cv2.waitKey(0)
# cv2.destroyAllWindows()
# print(np.shape(submap1))

In [3]:
train_label_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhuan448_angle"
train_pic_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhuan448_angle/picture"
train_submap_dir = "/home/ysk/catkin_ws/laser_pic_code/pic/regression_benhuan448_angle/submap"

def get_train_data_batch(num, pic_dir=train_pic_dir, label_dir=train_label_dir,submap_dir=train_submap_dir):
    train_txt = os.path.join(label_dir,"label.txt")
    count = len(open(train_txt,'r').readlines()) - 1    
    idx = np.arange(0, count)      
    np.random.shuffle(idx)                
    data_idx = idx[:num]
#     print(data_idx)
    
    labels = []
    images = []
    
    for i in range(num):
        with open(train_txt,'r') as file:
            s = file.readlines()
            image1 = skimage.data.imread(os.path.join(pic_dir, s[data_idx[i]].split(";")[0]))
            submap1 = cv2.imread(os.path.join(submap_dir,str(data_idx[i]) + '.pgm'))
            submap1 = cv2.resize(submap1,(448,448))
            submap1 = cv2.transpose(submap1)
            submap1 = cv2.flip(submap1, 1)
            img = cv2.merge([image1,submap1])
            images.append(img)

            label = s[data_idx[i]].split(";")[1:]
            label = list(map(float,label))
            labels.append(label)
    return np.asarray(images),np.asarray(labels)

num = 10
# for i in range(10):
train_images, train_labels = get_train_data_batch(num)
#     print(train_labels)
print(np.shape(train_images))
print(np.shape(train_labels))
print(train_labels)
# print(train_labels*100)

(10, 448, 448, 6)
(10, 3)
[[ 1.20145202e+01  1.61486020e+00  4.35688194e+01]
 [ 8.53054872e+00 -1.52672803e-02  3.61049877e+00]
 [-1.31765933e+01  3.81320659e+01  4.58146481e+01]
 [-7.59490129e+00  4.08376702e+01 -1.78374512e+02]
 [ 1.82988268e+01  3.31585331e+01 -4.38750893e+01]
 [ 1.01337654e+01  9.48796657e-02 -2.36536049e+00]
 [ 2.05375280e+01  2.18238879e+01  8.77467826e+01]
 [-2.29385749e+00  4.09209276e+01 -1.77453135e+02]
 [-2.05247972e+01  1.56634498e+01  9.37428051e+01]
 [-1.44638066e+01  3.67247033e+01 -1.34656029e+02]]


In [4]:
with tf.name_scope("input"):
    input_image = tf.placeholder("float", [None, 448,448,6],name='input_image')/255.*2.-1
#     input_image = tf.pad(image,np.array([[0, 0], [3, 3], [3, 3], [0, 0]]))
    y_ = tf.placeholder("float", [None, 3],name='y_') 

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")


W_conv0 = weight_variable([7, 7, 6, 64])
b_conv0 = bias_variable([64])

h_conv0 = tf.nn.leaky_relu(tf.nn.conv2d(input_image, W_conv0, strides=[1,1,1,1], padding="SAME") + b_conv0)
h_pool0 = max_pool_2x2(h_conv0)   #输出变成224*224*64 [None,224,224,64]

# #第一层卷积层
# #参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度

W_conv1 = weight_variable([7, 7, 64, 128])
b_conv1 = bias_variable([128])

h_conv1 = tf.nn.leaky_relu(conv2d(h_pool0, W_conv1) + b_conv1)
# h_conv1 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv1, strides=[1,2,2,1], padding="SAME") + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64   [None,112,112,64]

# #第一层卷积层
# #参数的前两维为卷积核的维度，第三个参数为当前层的深度，第四个为输出到层的深度
# W_conv1 = weight_variable([7, 7, 6, 64])
# b_conv1 = bias_variable([64])

# h_conv1 = tf.nn.relu(tf.nn.conv2d(input_image, W_conv1,strides=[1,2,2,1],padding="SAME") + b_conv1)
# h_pool1 = max_pool_2x2(h_conv1)   #输出变成112*112*64

#第二层卷积层
W_conv2 = weight_variable([3, 3, 128, 192])
b_conv2 = bias_variable([192])

h_conv2 = tf.nn.leaky_relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)  #输出变为56*56*192


#第三层卷积层
W_conv3 = weight_variable([3, 3, 192, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.leaky_relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)   #输出为28*28*128


#第四层卷积层
W_conv4 = weight_variable([3, 3, 128, 64])
b_conv4 = bias_variable([64])

h_conv4 = tf.nn.leaky_relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)   #输出变为14*14*64


#密集连接层1
W_fc1 = weight_variable([14*14*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool4, [-1, 14*14*64])
h_fc1 = tf.nn.leaky_relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)   #输出为96*1024

keep_prob = tf.placeholder("float")
#relu函数作用是求出max(h_fc1,0)
h_fc1_drop1 = tf.nn.dropout(h_fc1, keep_prob)   #输出为96*1024


#密集连接层2
W_fc2 = weight_variable([1024, 512])
b_fc2 = bias_variable([512])

h_fc2 = tf.nn.leaky_relu(tf.matmul(h_fc1_drop1, W_fc2) + b_fc2)

#relu函数作用是求出max(h_fc1,0)
h_fc1_drop2 = tf.nn.dropout(h_fc2, keep_prob)

#输出层softmax层
W_fc3 = weight_variable([512, 3])

y_conv = tf.matmul(h_fc1_drop2, W_fc3)
# tf.summary.histogram('y_conv',y_conv)

# batch_x = images[1:3]
# batch_y = labels[1:3]
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     print("shape of outputs is:",sess.run(tf.shape(y_conv),feed_dict={input_image:batch_x, 
#                                                                            y_:batch_y,keep_prob:1.0}))
#         print("learning rate is :",sess.run(learning_rate,feed_dict = {global_step:i}))


starter_learning_rate = 0.0003
steps_per_decay = 3000
decay_factor = 0.9

global_step = tf.Variable(tf.constant(0))
learning_rate = tf.train.exponential_decay(learning_rate = starter_learning_rate,
                                           global_step = global_step,
                                           decay_steps = steps_per_decay,
                                           decay_rate = decay_factor,
                                           staircase = True
                                           )

delta=tf.constant(1.25)
#huber损失函数
loss = tf.reduce_mean(tf.multiply(tf.square(delta),
                                          tf.sqrt(1. + tf.square((y_ - y_conv)/delta)) - 1.))

# cross_entropy = tf.reduce_sum(tf.square(y_-y_conv)) 
tf.summary.scalar("loss",loss)

train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss)
# train_step = tf.train.AdadeltaOptimizer(learning_rate).minimize(cross_entropy)
# train_step = tf.train.RMSPropOptimizer(learning_rate).minimize(cross_entropy)

# correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# tf.summary.scalar("accuracy", accuracy)
merged = tf.summary.merge_all()

batch_size = 4          
checkpointsPath = "./checkpoints/"
reload = False

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #保存训练好的模型一遍下次使用
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter("logs_dir/", sess.graph)
    
    if not os.path.exists(checkpointsPath):
        os.mkdir(checkpointsPath)
    
    if reload:
        checkPoint = tf.train.get_checkpoint_state(checkpointsPath)
        if checkPoint and checkPoint.model_checkpoint_path:
            saver.restore(sess,checkPoint.model_checkpoint_path)
            print("restored %s" % checkPoint.model_checkpoint_path)
        else:
            print("no checkpoint found!")
        
    for i in tqdm(range(100000)):
        batch_x,batch_y = get_train_data_batch(batch_size)
        batch_y = batch_y*1000
        summary,_ = sess.run([merged,train_step],feed_dict={global_step:i,
                                                            input_image:batch_x, 
                                                            y_:batch_y, 
                                                            keep_prob:0.8})
        writer.add_summary(summary, i)
        if (i % 1000) == 0 & (i != 0):
            print("save model")
            print("learning_rate is:\n",sess.run(learning_rate,feed_dict = {global_step:i}))
            print("real location is:\n",batch_y)
            print("prediction is:\n",sess.run(y_conv,feed_dict={input_image:batch_x, y_:batch_y,keep_prob:1.0}))
#             saver.save(sess, os.path.join(checkpointsPath,"saved_net.ckpt"), global_step=i)
            print("loss is:",sess.run(loss,feed_dict={input_image:batch_x, y_:batch_y, keep_prob:1.0}))
    print("done!")


  0%|          | 0/100000 [00:00<?, ?it/s]

save model
learning_rate is:
 0.0003
real location is:
 [[-1.51696841e+02  4.09584690e+04 -1.77542120e+05]
 [-1.52656074e+04  3.59238357e+04 -1.35359471e+05]
 [ 2.03574610e+04  2.91090618e+04 -8.56803649e+04]
 [-1.13988161e+04  1.61386346e+03  1.38558885e+05]]
prediction is:
 [[ -253918.64  -369651.75 -1177068.5 ]
 [ -295920.94  -380123.34 -1132698.5 ]
 [ -282159.03  -357545.12 -1148225.5 ]
 [ -280174.25  -385554.47 -1148778.5 ]]


  0%|          | 1/100000 [00:02<63:20:57,  2.28s/it]

loss is: 734679.94


  1%|          | 1001/100000 [04:24<8:23:57,  3.27it/s]

...

save model
learning_rate is:
 1.0301043e-05
real location is:
 [[   4361.08746478   41098.3118707  -177524.696329  ]
 [  20367.7612361    10335.4012906  -136800.993962  ]
 [ -20575.1015798    13238.939678    -90776.9626421 ]
 [   6011.58512397   40952.4132041   179376.303526  ]]
prediction is:
 [[   4760.6553   39196.973  -173292.25  ]
 [  20529.777    11471.343  -137719.89  ]
 [ -19992.346    12854.722   -89597.375 ]
 [   4461.966    41994.594   185480.47  ]]
loss is: 2039.3492


 99%|█████████▉| 99001/100000 [7:10:30<05:05,  3.27it/s]

save model
learning_rate is:
 9.270939e-06
real location is:
 [[  14548.3614373    36802.0650863   -44521.1412181 ]
 [ -16215.1250027     5944.55476491  136612.510056  ]
 [ -20704.5686066    16659.4116669   -89682.9507161 ]
 [ -15054.0602627    36134.0448937  -136295.077326  ]]
prediction is:
 [[  14828.919    37727.53    -46290.93  ]
 [ -15254.471     5228.3604  134514.2   ]
 [ -20267.275    16030.196   -84935.25  ]
 [ -13957.767    35329.402  -135945.5   ]]
loss is: 1541.7407


100%|██████████| 100000/100000 [7:14:50<00:00,  3.84it/s]

done!
